In [16]:
import requests,zipfile, io
import pandas as pd
import concurrent.futures

In [32]:
def download_bse_bhavcopy(date, save_location):
    date_str = date.strftime('%d%m%y')
    date_str_file = date.strftime('%d%m%Y')
    url = "http://www.bseindia.com/download/BhavCopy/Equity/EQ{}_CSV.zip".format(date_str)
    r = requests.get(url)
    
    if r.status_code == 200:
        z = zipfile.ZipFile(io.BytesIO(r.content))
        print(z.namelist())
        dt = f"EQ{date_str_file}.csv"
        df = pd.read_csv(z.open(f"EQ{date_str}.CSV"))
        df = df[df.SC_TYPE == 'Q']
        
        df = df.set_index('SC_CODE')
        df = df[['SC_NAME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'NO_OF_SHRS']]
        df['DATE']=date_str_file
        
        df.to_csv(f"{save_location}/{dt}")
    else:
        raise ValueError(f"Received Error code {r.status_code} while downloading for date ${date_str} from url ${url}")
    


In [33]:
def get_bse_bhavcopy(start=None, end=None, periods=5, save_location='.'):
    if not end:
        end=pd.datetime.today()

    if not start:
        datelist = pd.bdate_range(end=end, periods=periods).tolist()
    else:
        datelist = pd.bdate_range(start=start, end=end).tolist()
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_score = {}
        error_list = []

        for d in datelist:
            date_str = d.strftime('%d%m%y')
            future_score.update({executor.submit(download_bse_bhavcopy, d,save_location): date_str})

        for future in concurrent.futures.as_completed(future_score):
            date_str = future_score[future]
            try:
                future.result()
                print(f"{date_str} is done")
            except Exception as exc:
                print(f"{date_str} generated error: ${exc}")


In [34]:
d = get_bse_bhavcopy(periods=14, save_location='/Users/Udit/Dropbox/Watchlist/eoddata/bse')

['EQ181217.CSV']
251217 generated error: $Received Error code 404 while downloading for date $251217 from url $http://www.bseindia.com/download/BhavCopy/Equity/EQ251217_CSV.zip
181217 is done
['EQ201217.CSV']
201217 is done
['EQ261217.CSV']
261217 is done
['EQ221217.CSV']
221217 is done
['EQ121217.CSV']
['EQ141217.CSV']
['EQ281217.CSV']
121217 is done
141217 is done
['EQ291217.CSV']
281217 is done
['EQ151217.CSV']
['EQ211217.CSV']291217 is done

151217 is done
211217 is done
['EQ271217.CSV']
271217 is done
['EQ131217.CSV']
131217 is done
['EQ191217.CSV']
191217 is done
